## ADF Diagnostics In Jupyter
This notebook will run the Atmospheric Diagnostic Framework using the settings in a config.yaml file in your ADF directory. 

Note that it was developed to run on Cheyenne/Caspar JupyterHub *with the NPL (conda) kernel*

### Setup
#### Required packages

In [ ]:
import os.path
import sys

#### Paths

In [ ]:
# ADF Code path
# If it is in your cwd, set adf_code = local_path (initialized below)
# otherwise set adf_code appropriately

local_path = os.path.abspath('')
adf_code = local_path
#adf_code = "/glade/u/home/bundy/diag/ADF_top/ADF"

#set path to ADF lib (main code)
lib_path = os.path.join(adf_code,"lib")

#set path to ADF plotting scripts 
plotting_scripts_path = os.path.join(adf_code,"scripts","plotting")

#Add paths to python path:
sys.path.append(lib_path)
sys.path.append(plotting_scripts_path)

print(f"current working directory = {local_path}")
print(f"ADF path                  = {adf_code}")
print(f"ADF source code           = {lib_path}")
print(f"ADF plotting scripts      = {plotting_scripts_path}")


#### Paths to data that is input or produced by the ADF are set in the config.yaml file. 
To modify from the defaults, _edit that file_.

If there are errors here, it is likely due to path errors above



In [ ]:
#set path to config YAML file:
config_file=os.path.join(adf_code,"config_cam_baseline_example.yaml")

print(f"Will read settings from ",config_file)

In [ ]:
#import ADF diagnostics object
from adf_diag import AdfDiag

# If this fails, check your paths output in the cells above,
# and that you are running the NPL (conda) Kernel
# You can see all the paths being examined by un-commenting the following:
#sys.path

In [ ]:
#
# Initialize ADF object from the user-set config file
#
# Note that you will need to set 'user' in your config file. 
# The file makes some assumptions about output/working data directories 
# that can be checked by searching for where the variable 'user' is used.
#
# Any changes that are made to the config file will require re-running this cell
# 
adf = AdfDiag(config_file)
print(f"Reading settings from ",config_file)

check_user = adf.read_config_var("user")
err_msg = 'ERROR: You need to set user in the config.yaml to your user name'
assert check_user != 'USER-NAME-NOT-SET', f'{err_msg}'
print(f'user name set to: {check_user}')


In [ ]:
# Pull (and print) some info from the ADF object:
baseline_dict = adf.read_config_var("diag_cam_climo")
baseline_dict

In [ ]:
# More useful to get information with the variable settings (eg cam_case_name) replaced
# with their actual values:

#Grab a processed variable under "diag_basic_info":
print("ADF plots will be written to",adf.get_basic_info('cam_diag_plot_loc', required=True))

#Grab a processed variable under "diag_cam_climo":
print("case hist files will be read from",adf.get_cam_info('cam_hist_loc', required=True))

#Grab a processed variable under "diag_cam_baseline_climo":
print("baseline time series files will be read from",adf.get_baseline_info('cam_ts_loc', required=True))

#Finally, if you request a non-existent variable, and don't have "required=True", it will return None:
print("The number of pineapples in the config file is",adf.get_basic_info('pineapples'))

### ADF Standard Work Flow

In [ ]:
#Create model time series.
print(f"case ts files will be created in",adf.get_cam_info('cam_ts_loc', required=True))
adf.create_time_series()

In [ ]:
#Create model baseline time series (if needed):
if not adf.compare_obs:
    print(f"baseline ts files will be created in",adf.get_baseline_info('cam_ts_loc', required=True))
    adf.create_time_series(baseline=True)


In [ ]:
#Create model climatology (climo) files.
print(f"case climo files will be created in",adf.get_cam_info('cam_climo_loc', required=True))
adf.create_climo()

In [ ]:
#If a user is doing a model vs obs comparison, but
#no observations were found, then stop here:
if adf.compare_obs and not adf.var_obs_dict:
        print('ADF diagnostics has completed successfully.')
        sys.exit(0)
else:
    print('config file did not ask ADF to compare obs')

In [ ]:
#Regrid model climatology files to match either
#observations or CAM baseline climatologies.
#This call uses the "regridding_scripts" specified
#in the config file:
adf.regrid_climo()

In [ ]:
#Perform analyses on the simulation(s).
#This call uses the "analysis_scripts" specified in the
#config file:
adf.perform_analyses()

In [ ]:
#Create the plot requested in the plotting_scripts list in the config file:
plot_path = adf.get_basic_info('cam_diag_plot_loc')
print(f'plots written to {plot_path}')
adf.create_plots()

In [ ]:
#Create website.
if adf.create_html:
    adf.create_website()

### ADF Helpful Methods and Structures 

#### Demonstration of a few methods to get information from the ADF object

In [ ]:
basic_info_dict = adf.read_config_var("diag_basic_info")
print(basic_info_dict)

In [ ]:
baseline_dict = adf.read_config_var("diag_cam_baseline_climo")
print(baseline_dict)

In [ ]:
case_names = adf.get_cam_info("cam_case_name",required=True)
print(case_names)

In [ ]:
plot_type = basic_info_dict.get('plot_type', 'png')
plot_type

In [ ]:
case_climo_loc = adf.get_cam_info('cam_climo_loc', required=True)

# Variables used in a CAM v CAM (baseline comparison), not required because not used if compare_obs = True
data_name = adf.get_baseline_info('cam_case_name', required=False)
data_loc = adf.get_baseline_info("cam_climo_loc", required=False)

print(case_climo_loc)
print(data_name)
print(data_loc)

In [ ]:
var_list = adf.diag_var_list
print(var_list)

 #### Demonstrate how to check for a variable in the list 

In [ ]:
#Print all the variables
var_list

In [ ]:
# Check for a variable
var_name = 'PRECC'
assert var_name in var_list, f'Sorry, you need to include {var_name} to make this plot'

In [ ]:
# Check for a variable that isn't in the list
var_name = 'PRECT'
assert var_name in var_list, f'Sorry, you need to include {var_name} to make this plot'

# EXPECT AN ERROR for demonstration purposes! 
